In [1]:
import os, sys, glob
import spacy
import pandas as pd

from spacy.pipeline import EntityRuler

In [2]:
sys.path.append('..')

In [3]:
from cycontext.item_data import ItemData
from cycontext.tag_object import TagObject
from cycontext.context_component import ConTextComponent

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
nlp = spacy.load("en_core_web_sm")
_ = nlp.remove_pipe("ner")

# Overview
- Read in data
- Read in modifiers

# I. Read in Knowledge Base

## I.I Targets

In [6]:
targets_filepath = os.path.join(".", "pneumonia", "kb", "pneumonia_targets.tsv")
targets_df = pd.read_csv(targets_filepath, sep="\t")

In [7]:
targets_df.head()

,Lex,Type,Regex,Direction,Codes
0,pneumonia,EVIDENCE_OF_PNEUMONIA,pneumonias?,NaN,NaN
1,consolidation,EVIDENCE_OF_PNEUMONIA,NaN,NaN,NaN
2,infiltrate,EVIDENCE_OF_PNEUMONIA,infiltrat(e|es|ion),NaN,NaN
3,effusion,EVIDENCE_OF_PNEUMONIA,effusion,NaN,NaN
4,effusion,EVIDENCE_OF_PNEUMONIA,opacit(y|ies),NaN,NaN


In [8]:
target_patterns = []
for i, row in targets_df.fillna('').iterrows():
    literal = row["Lex"]
    category = row["Type"]
    regex = row["Regex"]
    
    if regex == '':
        pattern = [{"LOWER": literal}]
    else:
        pattern = [{"LOWER": {"REGEX": regex}}]
    target_patterns.append({"label": category, "pattern": pattern})


In [9]:
# Use the EntityRuler class for rule-based NER
ruler = EntityRuler(nlp, overwrite_ents=True)
ruler.add_patterns(target_patterns)

In [10]:
try:
    _ = nlp.remove_pipe("entity_ruler")
except:
    pass
nlp.add_pipe(ruler)

## I.II Modifiers

In [11]:
filepath = os.path.join(".", "pneumonia", "kb", "pneumonia_modifiers.json")

In [12]:
import json
with open(filepath) as f:
    modifier_data = json.load(f)

In [13]:
modifier_data["patterns"][:10]

[{'literal': 'are ruled out',
  'category': 'DEFINITE_NEGATED_EXISTENCE',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'be ruled out',
  'category': 'INDICATION',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'being ruled out',
  'category': 'INDICATION',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'can be ruled out',
  'category': 'DEFINITE_NEGATED_EXISTENCE',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'cannot be excluded',
  'category': 'AMBIVALENT_EXISTENCE',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'cannot totally be excluded',
  'category': 'PROBABLE_NEGATED_EXISTENCE',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'could be ruled out',
  'category': 'DEFINITE_NEGATED_EXISTENCE',
  'pattern': None,
  'rule': 'backward'},
 {'literal': 'examination',
  'category': 'INDICATION',
  'pattern': [{'LOWER': {'REGEX': '(examination|exam|study)'}}],
  'rule': 'backward'},
 {'literal': 'free',
  'category': 'DEFINITE_NEGATED_EX

In [14]:
item_data = []
for data in modifier_data["patterns"]:
    item = ItemData(**data)
    item_data.append(item)

In [15]:
item_data[:10]

[ItemData: [are ruled out, definite_negated_existence, None, backward],
 ItemData: [be ruled out, indication, None, backward],
 ItemData: [being ruled out, indication, None, backward],
 ItemData: [can be ruled out, definite_negated_existence, None, backward],
 ItemData: [cannot be excluded, ambivalent_existence, None, backward],
 ItemData: [cannot totally be excluded, probable_negated_existence, None, backward],
 ItemData: [could be ruled out, definite_negated_existence, None, backward],
 ItemData: [examination, indication, [{'LOWER': {'REGEX': '(examination|exam|study)'}}], backward],
 ItemData: [free, definite_negated_existence, None, backward],
 ItemData: [has been ruled out, definite_negated_existence, None, backward]]

# II. Read Data

In [16]:
txt_files = glob.glob(os.path.join(".", "pneumonia", "training_v2", "*.txt"))

In [17]:
texts = []
for file in txt_files:
    texts.append(open(file).read())

# III. Process with NLP

## III.II Add ConText

In [18]:
context = ConTextComponent(item_data, nlp)

In [19]:
try:
    nlp.add_pipe(context)
except:
    nlp.remove_pipe("context")
    nlp.add_pipe(context)

### III.I Custom sentence segmentation
The spaCy default works poorly with MIMIC data. Should eventually replace with PyRUSH.

In [20]:
from nltk.tokenize import PunktSentenceTokenizer

In [21]:
def set_custom_boundaries(doc, tokenizer):
    sent_spans = tokenizer.span_tokenize(doc.text)
    for token in doc:
        token.is_sent_start = False
    for (start, end) in sent_spans:
        sent = doc.char_span(start, end)
        sent[0].is_sent_start = True
    return doc

In [22]:
try:
    nlp.remove_pipe('set_custom_boundaries')
except:
    pass
nlp.add_pipe(set_custom_boundaries, before="parser")

In [23]:
tokenizer = PunktSentenceTokenizer()

In [24]:
cfg = {'set_custom_boundaries': {"tokenizer": tokenizer}}

In [25]:
doc = nlp(texts[12], component_cfg=cfg)

# for sent in doc.sents:
#     print("---", sent)
#     print()

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [Increased, historical]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]



## 3.3 Process documents

In [26]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x110febb38>),
 ('set_custom_boundaries',
  <function __main__.set_custom_boundaries(doc, tokenizer)>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x111175ac8>),
 ('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x1132f9c88>),
 ('context', <cycontext.context_component.ConTextComponent at 0x11723cbe0>)]

In [27]:
docs = list(nlp.pipe(texts, component_cfg=cfg))

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [increased, historical]

<TagObject> [excluded, definite_negated_existence]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR, conj]

<TagObject> [

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [prior study, historical]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [prior study, historical]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [prior study, historical]

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR, conj]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REASON FOR THIS EXAMINATION, indication]

<TagObject> [REAS

In [28]:
# for doc in docs:
#     print(doc.ents)
#     print(doc._.context_graph.modifiers)
#     print(doc._.context_graph.edges)
#     print()

# Visualization

In [29]:
from cycontext import viz

In [30]:
from spacy import displacy

In [31]:
doc = docs[12]

In [32]:
doc._.context_graph.edges

[(infiltrate, <TagObject> [Reason, indication]),
 (infiltrate, <TagObject> [REASON, indication]),
 (infiltrate, <TagObject> [REASON FOR THIS EXAMINATION, indication]),
 (opacities, <TagObject> [increased, historical]),
 (opacities, <TagObject> [increased, historical]),
 (bronchopneumonia, <TagObject> [could represent, probable_existence]),
 (opacities, <TagObject> [Increased, historical]),
 (opacities, <TagObject> [Increased, historical])]

In [33]:
viz.visualize_targets(doc
                     , colors ={"EVIDENCE_OF_PNEUMONIA": "orange",
                                          "DEFINITE_NEGATED_EXISTENCE": "blue",
                               "indication": "yellow"})

In [36]:
short_docs = list(nlp.pipe(["Reason for this examination: infiltrate",
                          "The patchy opacity in the left lower lobe could represent bronchopneumonia"], 
                           component_cfg=cfg))

<TagObject> [Reason for, conj]

<TagObject> [Reason for this examination, indication]

<TagObject> [Reason for this examination, indication]



In [37]:
viz.visualize_modifiers(short_docs[1])